In [244]:
import time

start_time = time.time()

import os.path as op
import pandas as pd
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as pyplot


from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve,auc
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.contrib import rnn

scaler_small = MinMaxScaler((-1,1))
scaler_large = MinMaxScaler((-3,3))

#path_to_data = op.relpath("/modules/cs342/Assignment2/")
path_to_data = "./data"

light_curves = pd.read_csv(path_to_data + "/training_set.csv")
metadata     = pd.read_csv(path_to_data + "/training_set_metadata.csv")

target = metadata["target"]
metadata = metadata.drop("target", axis=1)

metadata = metadata.drop(["ra", "decl", "gal_l", "gal_b", "distmod"], axis=1)

kf = KFold(n_splits=5, shuffle=True)

bands = ["u_band", "g_band", "r_band", "i_band", "z_band", "y_band"]
classes= [6,15,16,42,52,53,62,64,65,67,88,90,92,95,99]
rev_classes = dict(zip(classes, range(15)))

target_matrix = np.zeros((len(target), len(classes)))
for i, value in enumerate(target):
    target_matrix[i,rev_classes[value]] = 1

# drop flux error and detected flag for now
light_curves = light_curves.drop(["flux_err", "detected"], axis=1)

print time.time() - start_time

0.80287694931


In [245]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

In [290]:
training_set = metadata.copy()

start_time = time.time()

def get_seq(light_curves):
    curves_seq = []

    for obj_id, group_obj in light_curves.groupby(["object_id"]):
        first_day = group_obj["mjd"].iat[0]
        times = np.sort(list(set([int(timespot-first_day) for timespot in group_obj["mjd"]])))
        #times = np.arange(0,length_of_domain)*gap_between_intervals
        fluxes = np.zeros((6,len(times)))

        for band_no, group in group_obj.drop("object_id", axis=1).groupby(["passband"]):
            fluxes[band_no,0] = group["flux"].iat[0]
            exam_index = 1
            for flux,this_time in zip(group["flux"], group["mjd"]):
                if this_time-first_day >= times[exam_index]:
                    fluxes[band_no,exam_index] = flux
                    while this_time-first_day >= times[exam_index]:
                        if exam_index+1 == len(times): break
                        exam_index+=1
                        fluxes[band_no,exam_index] = fluxes[band_no-1, exam_index]
                        
        curves_seq.append(np.expand_dims(fluxes.T,2))
    return curves_seq
    
curves_seq = get_seq(light_curves)
    
print time.time() - start_time

31.2327530384


In [292]:
model = Sequential()
model.add(LSTM(15, batch_input_shape=(None,None,6)))
model.add(Activation("softmax"))

model.compile("adam", loss="categorical_crossentropy")

In [293]:
start_time = time.time()

partition = len(curves_seq)*3/4
    
train_curves, test_curves = curves_seq[:partition], curves_seq[partition:]
train_target, test_target = target_matrix[:partition], target_matrix[partition:]

model.fit(train_curves, train_target)

print time.time()-start_time

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 5886 arrays: [array([[[    6.878784],
        [ -816.434326],
        [ -544.810303],
        [ -471.385529],
        [ -388.984985],
        [ -355.88678 ]],

       [[    6.878784],
        [-1061.457031],
     ...